In [3]:
from abc import ABC, abstractmethod

# Command Interface
class Command(ABC):
    @abstractmethod
    def execute(self):
        pass

    @abstractmethod
    def undo(self):
        pass

# Receiver
class ShoppingCart:
    def __init__(self):
        self.items = []

    def add_item(self, item):
        self.items.append(item)
        print(f"ShoppingCart: Added {item} to the cart.")

    def remove_item(self, item):
        if item in self.items:
            self.items.remove(item)
            print(f"ShoppingCart: Removed {item} from the cart.")
        else:
            print(f"ShoppingCart: Item {item} not found in the cart.")

    def checkout(self):
        if not self.items:
            print("ShoppingCart: Cart is empty. Nothing to checkout.")
            return
        print(f"ShoppingCart: Checking out with items: {', '.join(self.items)}")
        self.items.clear()

# Concrete Commands
class AddToCartCommand(Command):
    def __init__(self, cart: ShoppingCart, item: str):
        self.cart = cart
        self.item = item

    def execute(self):
        self.cart.add_item(self.item)

    def undo(self):
        self.cart.remove_item(self.item)

class RemoveFromCartCommand(Command):
    def __init__(self, cart: ShoppingCart, item: str):
        self.cart = cart
        self.item = item

    def execute(self):
        self.cart.remove_item(self.item)

    def undo(self):
        self.cart.add_item(self.item)

class CheckoutCommand(Command):
    def __init__(self, cart: ShoppingCart):
        self.cart = cart
        self.checked_out_items = []

    def execute(self):
        self.checked_out_items = self.cart.items.copy()
        self.cart.checkout()

    def undo(self):
        if self.checked_out_items:
            for item in self.checked_out_items:
                self.cart.add_item(item)
            print("CheckoutCommand: Checkout undone. Items restored to the cart.")
        else:
            print("CheckoutCommand: No checkout to undo.")

# Invoker
class User:
    def __init__(self):
        self.cart = ShoppingCart()
        self.history = []

    def execute_command(self, command: Command):
        command.execute()
        self.history.append(command)

    def undo_last_command(self):
        if not self.history:
            print("User: No commands to undo.")
            return
        command = self.history.pop()
        command.undo()
        print("User: Undid the last command.")


In [5]:
# Client
if __name__ == "__main__":
    user = User()

    # Creating command objects
    add_item_1 = AddToCartCommand(user.cart, "Laptop")
    add_item_2 = AddToCartCommand(user.cart, "Smartphone")
    add_item_3 = AddToCartCommand(user.cart, "TV")
    remove_item = RemoveFromCartCommand(user.cart, "Laptop")
    checkout = CheckoutCommand(user.cart)

    # Executing commands
    user.execute_command(add_item_1)       # Adds Laptop
    user.execute_command(add_item_2)       # Adds Smartphone
    user.execute_command(add_item_3)       # Adds TV
    user.execute_command(remove_item)       # Removes Laptop
    user.execute_command(checkout)          # Checks out

    # Undoing commands
    user.undo_last_command()                # Undo Checkout
    user.undo_last_command()                # Undo Remove Laptop
    user.undo_last_command()                # Undo Add TV
    user.undo_last_command()                # Undo Add Smartphone
    user.undo_last_command()                # Undo Add Laptop
    user.undo_last_command()                # No commands to undo

ShoppingCart: Added Laptop to the cart.
ShoppingCart: Added Smartphone to the cart.
ShoppingCart: Added TV to the cart.
ShoppingCart: Removed Laptop from the cart.
ShoppingCart: Checking out with items: Smartphone, TV
ShoppingCart: Added Smartphone to the cart.
ShoppingCart: Added TV to the cart.
CheckoutCommand: Checkout undone. Items restored to the cart.
User: Undid the last command.
ShoppingCart: Added Laptop to the cart.
User: Undid the last command.
ShoppingCart: Removed TV from the cart.
User: Undid the last command.
ShoppingCart: Removed Smartphone from the cart.
User: Undid the last command.
ShoppingCart: Removed Laptop from the cart.
User: Undid the last command.
User: No commands to undo.
